In [1]:
# The variational quantum eigensolver (VQE)

# first install
# pip install pyscf # it enable us to do chemistry calculations

import qiskit
qiskit.__qiskit_version__

{'qiskit-terra': '0.19.2', 'qiskit-aer': '0.10.3', 'qiskit-ignis': '0.7.0', 'qiskit-ibmq-provider': '0.18.3', 'qiskit-aqua': None, 'qiskit': '0.34.2', 'qiskit-nature': None, 'qiskit-finance': None, 'qiskit-optimization': None, 'qiskit-machine-learning': None}

In [2]:
import numpy as np
import pylab
import copy
from qiskit import BasicAer

# from qiskit.aqua import aqua_globals, QuantumInstance
from qiskit.utils import QuantumInstance

# from qiskit.aqua.algoritms import NumpyMinimumEigensolver, VQE
from qiskit.algorithms import VQE, NumPyMinimumEigensolver

# from qiskit.aqua.components.optimizers import SLSQP
from qiskit.algorithms.optimizers import SLSQP

from qiskit.chemistry.components.initial_states import HartreeFock
from qiskit.chemistry.components.varuiational_forms import UCCSD
from qiskit.chemistry.drivers import PySCFDriver
from qiskit.chemistry.core import Hamiltonian, QubitMappingType

ModuleNotFoundError: No module named 'qiskit.chemistry'

In [ ]:
molecule = 'H .0 .0 -{0}; Li .0 .0 {0}'
distances = np.arrange(0.5, 4.25, 0.25)
vqe_energies = []
hf_energies []
exact_energies = []

In [ ]:
for i,d in enumerate(distances):
    print('step', i)
    
    #set up experiment
    driver = PySCFDriver(molecule.format(d/2), basis='sto3g')
    qmolecule = driver.run()
    operator = Hamiltonian(qubit_mapping=QubitMappingType.PARITY,
                           two_qubit_reduction=True,freeze_core=True,
                           orbit_reduction=[-3, -2])
    qubit_op, aux_ops = operator.run(qmolecule)
    
    #exact classical result
    exact_result = NumPyMinimumEigensolver(qubit_op,aux_operators=aux_ops)
    exact_result = operator.process_algorithm_result(exact_result)
    
    #VQE
    optimizer = SLSQP(maxiter=1000)
    initial_state = HartreeFock(operator.molecule_info['num_orbitals'],
                               operator_molecule_info['num_particles'],
                               qubit_mapping=operator._qubit_mapping,
                               two_qubit_reduction=operator._two_qubit_reduction)
    var_form = UCCSD(num_orbitals=operator.molecule_info['num_orbitals'],
                    num_particles=operator.molecule_info['num_particles'],
                    initial_state=initial_state,
                    qubit_mapping=operator._qubit_mapping,
                    two_qubit_reduction=operator._two_qubit_reduction)
    algo = VQE(qubit_op, var_form, optimizer, aux_operators=aux_ops)
    
    vqe_result = algo.run(QuantumInstance(BasicAer.get_backend('statevector_similator')))
    vqe_result = operator.process_algorithm_result(vqe_result)
    
    exact_energies.append(exact_result.energy)
    vqe_energies.append(vqe_result.energy)
    hf_energies.append(vqe_result.hertree_fock_energy)
    

In [ ]:
pylab.plot(distance,hf_energies, label='Hartree-Fork')
pylab.plot(distance,vqe_energies, 'o', label='VQE')
pylab.plot(distance,exact_energies, 'x', label='Exact')

pylab.xlabel('Interatomic distance')
pylab.ylabel('Energy')
pylab.title('LiH Ground State Energy')
pylab.Legend(loc='upper right')